In [ ]:
import imageio
import matplotlib.animation as ani
import matplotlib.cm as cmx
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import numpy as np

from matplotlib.patches import Ellipse
from PIL import Image
from sklearn import datasets
from sklearn.cluster import KMeans

Let us now implement the Gaussian density function. Ahem... I know you can use numpy functions for that, but I believe it is actually interesting to see how things work internally. Our aim is to create a function that implements this:

\begin{equation}
\large
p(\mathbf x | \mathbf\mu, \mathbf\Sigma) = \frac 1 {({2\pi})^{n/2}|\Sigma|^{1/2}}\exp\left(-\frac 1 2 (\mathbf x -\mathbf\mu)^T\mathbf\Sigma^{-1}(\mathbf x -\mathbf\mu)\right)
\end{equation}


In [ ]:
#x is input with three columns
x0 = np.array([[0.05, 1.413, 0.212], [0.85, -0.3, 1.11], [11.1, 0.4, 1.5], [0.27, 0.12, 1.44], [88, 12.33, 1.44]])
#mu=mean with three arrays
mu = np.mean(x0, axis=0)
print("\n Mean \n",mu)
cov = np.dot((x0 - mu).T, x0 - mu) / (x0.shape[0] - 1)
print("\n Covaiance \n",cov)

n = x0.shape[1]
diff = (x0 - mu).T
y = np.diagonal(1 / ((2 * np.pi) ** (n / 2) * np.linalg.det(cov) ** 0.5) * np.exp(-0.5 * np.dot(np.dot(diff.T, np.linalg.inv(cov)), diff))).reshape(-1, 1)
print("\n This Gaussain distribution is for three dimensions\n")
print("\n probability= \n",y)


 Mean 
 [20.054   2.7926  1.1404]

 Covaiance 
 [[1.46429833e+03 2.02328512e+02 7.59124800e+00]
 [2.02328512e+02 2.88241988e+01 6.42787700e-01]
 [7.59124800e+00 6.42787700e-01 2.92920800e-01]]

 This Gaussain distribution is for three dimensions


 probability= 
 [[0.00159853]
 [0.00481869]
 [0.00276259]
 [0.0014309 ]
 [0.00143998]]


# For the purposes of this exercise, we will be using the Iris dataset, which is probably already familiar to you. We can easily obtain it by using the __load_iris__ function provided by sklearn:

In [ ]:
iris = datasets.load_iris()
X = iris.data
X[:20]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3]])

# __Step 1__

This is the initialization step of the GMM. At this point, we must initialise our parameters $\pi_k$, $\mu_k$, and $\Sigma_k$. In this case, we are going to use the results of KMeans as an initial value for $\mu_k$, set $\pi_k$ to one over the number of clusters and $\Sigma_k$ to the identity matrix. We could also use random numbers for everything, but using a sensible initialisation procedure will help the algorithm achieve better results.

In [ ]:
n_clusters=3
clusters = []
idx = np.arange(X.shape[0])
    
# We use the KMeans centroids to initialise the GMM
    
kmeans = KMeans(n_clusters).fit(X)
mu_k = kmeans.cluster_centers_
    
for i in range(n_clusters):
    clusters.append({
            'pi_k': 1.0 / n_clusters,
            'mu_k': mu_k[i],
            'cov_k': np.identity(X.shape[1], dtype=np.float64)
        })
    
print("\n Clusters\n",type(clusters))
#print("\n Clusters\n",clusters)
#print(*clusters,sep='\n')

for elem in clusters:
    print("\n \n",elem )



 Clusters
 <class 'list'>

 
 {'pi_k': 0.3333333333333333, 'mu_k': array([5.9016129 , 2.7483871 , 4.39354839, 1.43387097]), 'cov_k': array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])}

 
 {'pi_k': 0.3333333333333333, 'mu_k': array([5.006, 3.428, 1.462, 0.246]), 'cov_k': array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])}

 
 {'pi_k': 0.3333333333333333, 'mu_k': array([6.85      , 3.07368421, 5.74210526, 2.07105263]), 'cov_k': array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])}


**Step 2 (Expectation step)**

We should now calculate $\gamma(z_{nk})$. We can achieve this by means of the following expression:

\begin{equation}
\large
\gamma{(z_{nk})}=\frac {\pi_k\mathcal N(\mathbf x_n| \mathbf\mu_k, \mathbf\Sigma_k)}{\sum_{j=1}^K\pi_j\mathcal N(\mathbf x_n| \mathbf\mu_j, \mathbf\Sigma_j)}
\end{equation}

For convenience, we just calculate the denominator as a sum over all terms in the numerator, and then assign it to a variable named __totals__

In [ ]:
def gaussian(X, mu, cov):
    n = X.shape[1]
    diff = (X - mu).T
    return np.diagonal(1 / ((2 * np.pi) ** (n / 2) * np.linalg.det(cov) ** 0.5) * np.exp(-0.5 * np.dot(np.dot(diff.T, np.linalg.inv(cov)), diff))).reshape(-1, 1)

In [ ]:
#global gamma_nk, totals
N = X.shape[0]
K = len(clusters)
totals = np.zeros((N, 1), dtype=np.float64)
gamma_nk = np.zeros((N, K), dtype=np.float64)

for k, cluster in enumerate(clusters):
    pi_k = cluster['pi_k']
    mu_k = cluster['mu_k']
    cov_k = cluster['cov_k']
    #numpy.ravel flattens the array into 1-D array
    prod=(pi_k * gaussian(X, mu_k, cov_k))
    print("\n prod shape\n ",prod.shape)
    #numpy.ravel flattens the array into 1-D array
    prod2=prod.ravel()
    print("\n prod2 shape\n ",prod2.shape)
    #column is added to gamma_nk
    gamma_nk[:, k] = prod2

totals = np.sum(gamma_nk, 1)
print("\n totals shape \n",totals.shape)
#since totals has only rows we use np.expand_dims to include column
gamma_nk /= np.expand_dims(totals, 1)
print("\n gamma_nk shape \n",gamma_nk.shape)
print("\n gamma_nk  \n",type(gamma_nk))
print("\n gamma_nk  \n",gamma_nk[0:10,:])



 prod shape
  (150, 1)

 prod2 shape
  (150,)

 prod shape
  (150, 1)

 prod2 shape
  (150,)

 prod shape
  (150, 1)

 prod2 shape
  (150,)

 totals shape 
 (150,)

 gamma_nk shape 
 (150, 3)

 gamma_nk  
 <class 'numpy.ndarray'>

 gamma_nk  
 [[9.97084205e-01 2.91301279e-03 2.78180471e-06]
 [9.96578648e-01 3.41905629e-03 2.29549986e-06]
 [9.98135034e-01 1.86400069e-03 9.65448472e-07]
 [9.96724020e-01 3.27402499e-03 1.95516665e-06]
 [9.97508245e-01 2.48952138e-03 2.23377617e-06]
 [9.91157572e-01 8.82072077e-03 2.17074167e-05]
 [9.97753409e-01 2.24521378e-03 1.37672525e-06]
 [9.96177919e-01 3.81840714e-03 3.67382242e-06]
 [9.97657998e-01 2.34105547e-03 9.46925056e-07]
 [9.96195703e-01 3.80146566e-03 2.83102823e-06]]


**Step 3 (Maximization step):**

Let us now implement the maximization step. Since $\gamma(z_{nk})$ is common to the expressions for $\pi_k$, $\mu_k$ and $\Sigma_k$, we can simply define:

\begin{equation}
\large
N_k=\sum_{n=1}^N\gamma({z_{nk}})
\end{equation}

And then we can calculate the revised parameters by using:

\begin{equation}
\large
\pi_k^*=\frac {N_k} N
\end{equation}


\begin{equation}
\large
\mu_k^*=\frac 1 {N_k} \sum_{n=1}^N\gamma({z_{nk}})\mathbf x_n
\end{equation}


\begin{equation}
\large
\Sigma_k^*=\frac 1 {N_k} \sum_{n=1}^N\gamma({z_{nk}})(\mathbf x_n-\mathbf\mu_k)(\mathbf x_n-\mathbf\mu_k)^T
\end{equation}

Note: To calculate the covariance, we define an auxiliary variable __diff__ that contains $(x_n-\mu_k)^T$.

In [ ]:
N = float(X.shape[0])

for k, cluster in enumerate(clusters):
    gamma_k = np.expand_dims(gamma_nk[:, k], 1)
    N_k = np.sum(gamma_k, axis=0)
    
    pi_k = N_k / N
    mu_k = np.sum(gamma_k * X, axis=0) / N_k
    cov_k = (gamma_k * (X - mu_k)).T @ (X - mu_k) / N_k
    
    cluster['pi_k'] = pi_k
    cluster['mu_k'] = mu_k
    cluster['cov_k'] = cov_k

print("\n Clusters\n",clusters[0])
print("\n Clusters\n",clusters[1])
print("\n Clusters\n",clusters[2])


 Clusters
 {'pi_k': array([0.34077089]), 'mu_k': array([5.01175962, 3.40296744, 1.51313818, 0.26794469]), 'cov_k': array([[ 0.12319189,  0.09038184,  0.0300872 ,  0.01556953],
       [ 0.09038184,  0.16169368, -0.03585906, -0.01113107],
       [ 0.0300872 , -0.03585906,  0.13239249,  0.04920678],
       [ 0.01556953, -0.01113107,  0.04920678,  0.02930876]])}

 Clusters
 {'pi_k': array([0.36962874]), 'mu_k': array([5.96205682, 2.77674085, 4.48948813, 1.48307989]), 'cov_k': array([[0.26299967, 0.07845149, 0.22896555, 0.08978328],
       [0.07845149, 0.09463116, 0.08024942, 0.04960538],
       [0.22896555, 0.08024942, 0.41856938, 0.1970684 ],
       [0.08978328, 0.04960538, 0.1970684 , 0.131168  ]])}

 Clusters
 {'pi_k': array([0.28960037]), 'mu_k': array([6.67030916, 3.00875952, 5.46588637, 1.93313508]), 'cov_k': array([[0.35572165, 0.06886176, 0.32344008, 0.07669752],
       [0.06886176, 0.09716685, 0.06696857, 0.0486227 ],
       [0.32344008, 0.06696857, 0.46920563, 0.15609039],
     

Let us now determine the log-likelihood of the model. It is given by:

\begin{equation}
\large
\ln p(\mathbf X)=\sum_{n=1}^N\ln\sum_{k=1}^K\pi_k\mathcal N(\mathbf x_n|\mu_k,\Sigma_k)
\end{equation}

However, the second summation has already been calculated in the __expectation_step__ function and is available in the __totals__ variable. So we just make use of it.

In [ ]:
sample_likelihoods = np.log(totals)
sample_likelihoods_sum=np.sum(sample_likelihoods)
print("\n sample_likelihoods shape \n ",sample_likelihoods.shape)
print("\n sample_likelihoods \n ",sample_likelihoods)

print("\n sample_likelihoods_sum \n ",sample_likelihoods_sum)


 sample_likelihoods shape 
  (150,)

 sample_likelihoods 
  [-4.78143637 -4.8711292  -4.85948971 -4.90907506 -4.78966156 -4.99467467
 -4.8583073  -4.77272702 -5.09801167 -4.84134487 -4.88628665 -4.80226568
 -4.89697316 -5.19061779 -5.28638908 -5.49554203 -4.98560141 -4.78146846
 -5.10311906 -4.84656104 -4.87192936 -4.82402424 -4.97881642 -4.83608639
 -4.88264386 -4.86982852 -4.78965261 -4.79327672 -4.79313883 -4.85326313
 -4.8546983  -4.85801432 -5.02782603 -5.19475756 -4.83126494 -4.83393465
 -4.91003946 -4.80533057 -5.06212185 -4.77676857 -4.78931331 -5.54991948
 -4.99672941 -4.84145336 -4.9427246  -4.88702781 -4.8542772  -4.88346865
 -4.85226936 -4.78249845 -4.85073253 -4.6388231  -4.68230036 -4.9673291
 -4.57764834 -4.64437717 -4.60615079 -5.76823049 -4.65926844 -5.06772478
 -5.83571131 -4.6613486  -5.01178888 -4.53493888 -5.04925456 -4.7690772
 -4.67785817 -4.81736981 -4.7403121  -4.94919443 -4.61819795 -4.74779653
 -4.57467043 -4.59231281 -4.68025415 -4.70293444 -4.67276167 -4.5

In [ ]:
from sklearn.mixture import GaussianMixture

gmm = GaussianMixture(n_components=n_clusters, max_iter=50).fit(X)
gmm_scores = gmm.score_samples(X)

print('Means by sklearn:\n', gmm.means_)
print('Means by our implementation:\n', np.array([cluster['mu_k'].tolist() for cluster in clusters]))
print('Scores by sklearn:\n', gmm_scores[0:20])
print('Scores by our implementation:\n', sample_likelihoods.reshape(-1)[0:20])

Means by sklearn:
 [[5.91697517 2.77803998 4.20523542 1.29841561]
 [5.006      3.428      1.462      0.246     ]
 [6.54632887 2.94943079 5.4834877  1.98716063]]
Means by our implementation:
 [[5.01175962 3.40296744 1.51313818 0.26794469]
 [5.96205682 2.77674085 4.48948813 1.48307989]
 [6.67030916 3.00875952 5.46588637 1.93313508]]
Scores by sklearn:
 [ 1.57050082  0.73787138  1.14436656  0.92913238  1.411028   -0.09451903
  0.05266884  1.62442195  0.27082378  0.16706624  0.83489877  0.77168582
  0.29597841 -1.79224582 -3.41557928 -2.10529279 -1.12995447  1.47503579
 -0.84612536  0.97699215]
Scores by our implementation:
 [-4.78143637 -4.8711292  -4.85948971 -4.90907506 -4.78966156 -4.99467467
 -4.8583073  -4.77272702 -5.09801167 -4.84134487 -4.88628665 -4.80226568
 -4.89697316 -5.19061779 -5.28638908 -5.49554203 -4.98560141 -4.78146846
 -5.10311906 -4.84656104]




# MATH IN EM ALGORITHM



## 1.Probabilities
For iris dataset assume there are three classes

For each of the 150 samples we can find probability as follows:

$p(x) =p(x|G_1)P(G_1)+p(x|G_2)P(G_2)+p(x|G_3)P(G_3)$

Where 

 1. $G_1,\,G_2,\,G_3$ are three clusters or groups or mixture components, 
 2. x is sample
 3. $P(G_1),\,P(G_2),\,P(G_3)$ are priors ,cluster proportions or  mixture proportions
 4. $p(x|G_1),\,p(x|G_2),\,p(x|G_3)$ are sample cluster densities or component densities

$ \therefore p(x) =\Sigma^k_{i=1}p(x|G_i)P(G_i)\, here\,k=1,2,3$

The sample can be written as $X={\{x^t\}}_t$ 

For Iris dataset t=1,2,3...150  and sample is ***iid=independent and identically distributed***

##2.Assumption:
 The component densities $p(x|G_i)$ are multivariate Gaussian

 $\therefore p(x|G_i)\sim \mathcal{N}(\mu_i,Σ_i)$

 The parameters to be Estimated are 

 $Φ={\{p(x|G_i),\mu_i,Σ_i\}}^k_{i=1}$

## 3.Loglikelihood:

$L(Φ|X) = log\prod_tp(x^t |Φ)=\sum_tlog\sum^k_{i=1}p(x^t|G_i)P(G_i)$

###procedure

1. For each sample calculate k cluster probabilities Sum and take log
2. Sum all t samples probabilities log calculated in step1

## 4.Expectation -Maximisation Algorithmn

Unsupervised Iris dataset  problem involves two sets of random variables of which one,***X(inputs), is observable*** and the other, ***Z(clusters), is hidden.*** 

The goal of the algorithm is to find the parameter vector $Φ $ to maximize the likelihood of the joint distribution of X and Z, the complete likelihood $\mathcal{L_c}(Φ|X,Z)$.

Since the Z values are not observed, we cannot work directly with the
complete data likelihood $\mathcal{L_c} $;

***1.Expectation***

Instead, we work with its expectation $E$ given $X$and the current parameter values $Φ^l$ , where ***l indexes iteration***.

This is the expectation (E) step of the algorithm.

E-step : $E(Φ|Φ^l) = E[Lc(Φ|X,Z)|X,Φ^l]$

***2.Maximisation***

 Then in the maximization
(M) step, we look for the new parameter values, $Φ^{l+1}$, that maximize this

M-step :$ Φ^{l+1}= argmax_ΦE(Φ|Φ^l)$

To find argmax we perform number of iterations(also called as epochs)

***Procedure***
1. E step :estimate Clusters(Z) for each sample
2. M step :From estimated Clusters(Z) in step1 we find cluster probabilities of Samples
3. Iterate 1 and 2 steps number of times

***EM  Algorithmn Similarities with K-Means***
1. E step :Estimate Clusters based on centroids
2. M step :From estimated Clusters  in step1 we find means of Clusters samoples
3. Iterate 1 and 2 steps number of times



## 5 Derivation of Expectation-Step formula

***I. Find Joint density of X and Z***

We define a vector of ***indicator variables*** $z^t = \{z^t_1, . . . , z^t_k\}$

 where $z^t_i= 1$ if $x^t$ belongs to cluster $G_i $, and 0 otherwise.

 Example:
 1. For iris dataset samples=150,clusters=3
 2. So no of Indicator varaibles=150X3


 ***z is a multinomial distribution*** from k categories with prior probabilities ***$π_i $***, shorthand for $P(G_i)$

 Then
$P(z^t) =\prod ^k_{i=1}π^{z^t_i}_i $

The likelihood of an observation $x^t$ is equal to its probability specified by
the component that generated it:

$p(x^t |z^t) =\prod^k_{i=1}p_i(x^t)^{z^t_i}$

$p_i(x^t)$ is shorthand for $p(x^t|G_i)$

 ***The joint density is***

$p(x^t , z^t) = P(z^t)p(x^t |z^t)$ ....(1)


***II. Likelihood of the sample X***

and the complete data likelihood of the iid sample X is

$Lc(Φ|X,Z) $=log of product of joint probability(X,Z)

$Lc(Φ|X,Z) = log\prod_t p(x^t , z^t |Φ)$

$Lc(Φ|X,Z)=\sum_t log\ p(x^t , z^t |Φ)$ ***(used in colab calculations)***

substitute ***joint density of X,Z*** from  equ...(1)

$Lc(Φ|X,Z)=\sum_t log\ P(z^t |Φ) + log\ p(x^t|z^t,Φ)$

After   bringing  out common indicator variable $z^t_i$

$Lc(Φ|X,Z)=\sum_t \sum_iz^t_i [logπ_i + log \ p_i(x^t |Φ)]$......(2)


***III. Estimation Step***

***IIIa.Parameter Estimation***

$E(Φ|Φ^l) ≡ E[log P(X,Z)|X,Φ^l]$

$E(Φ|Φ^l)= E[Lc(Φ|X,Z)|X,Φ^l)]$

substituting equ...(2)

$E(Φ|Φ^l)=\sum_t\sum_iE[z^t_i|X,Φ^l][logπ_i + log \ p_i(x^t |Φ^l)]$ ....(3)

***IIIB.Estimation of Z***

$E[z^t_i|X,Φ^l] = E[z^t_i|x^t ,Φ^l]$  where $x^t$ are i.i.d

$= P(z^t_i= 1|x^t ,Φ^l) \ $ where $ \ z^t_i\ $ is a 0/1 random variable

$=\frac{p(x^t |z^t_i= 1,Φ^l)P(z^t_i= 1|Φ^l)}{p(x^t |Φ^l)} \ $ ***Bayes’ rule***

$= \frac{p_i(x^t |Φ^l)π_i}{\Sigma_j p_j (x^t |Φ^l)π_j}$


$= \frac{p(x^t|G_i ,Φ^l)P(G_i)}{\sum_j p(x^t|G_j ,Φ^l)P(G_j} $
 

$= P(G_i|x^t ,Φ^l) ≡ h^t_i$ ....(4)

We see that the expected value of the hidden variable, $E[z^t_
i ]$, is the posterior probability that $x^t$ is generated by component $G_i$ .

 Because this is a probability, it is between 0 and 1 and is a “soft” label, as opposed to the 0/1 “hard” label of k-means.


 ***IV.Maximisation Step***

 We maximize E to get the next set of parameter values $Φ^{l+1}$:


$Φ^{l+1} = argmax_ΦE(Φ|Φ^l)$

From substituting equ...(4) in (3)

$E(Φ|Φ^l) =\sum_t\sum_i h^t_i[log π_i + log p_i(x^t |Φ^l)]$

***the above is parameter Estimate equation***

$=\sum_t\sum_ih^t_i log π_i +\sum_t\sum_ih^t_ilog \ p_i(x^t |Φ^l)$


The second term is independent of $π_i$ and using the constraint that
$\sum_i π_i = 1$ as the Lagrangian, we solve for($∇π_i$=differentiate w.r.t $\pi_i$)


$∇π_i\sum_t\sum_i h^t_i logπ_i − λ(\sum_iπ_i − 1) = 0$

and get


$π_i =\frac{\sum_t h^t_i}{N}$  ..........(5)


Similarly, the first term of parameter Estimate equation is independent of the components and can be dropped while estimating the parameters of the components.
We solve for($∇_Φ$=differentiate w.r.t $Φ$)

$∇_Φ\sum_t\sum_i h^t_i \log p_i(x^t |Φ) = 0$

If we assume Gaussian components, $ˆp_i(x^t |Φ) ∼ N(m_i , S_i),$ the M-step
is

By substituting gaussain equation in equation above we get

$∇_{m_i}\sum_t\sum_i h^t_i \log p_i(x^t |Φ)=∇_{m_i}\sum_t\sum_i h^t_i\frac{(\frac{-1}{2})(x^t -m_i)^T\Sigma^{-1}(x^t-m_i)}{2\pi^{\frac{N}{2}}|\Sigma|^{\frac{1}{2}}} $

By differentiating w.r.t $\mu$ we get

$\Sigma_th^t_i(x^t-m_i)=0$

$m^{l+1}_i=\frac{\sum_t h^t_ix^t}{\sum_t h^t_i}$ ..............(6)

By differentiating w.r.t $ S $(COVARIANCE)  we get

$∇_{\Sigma}\sum_t\sum_i h^t_i \log p_i(x^t |Φ)=∇_{\Sigma}\sum_t\sum_i h^t_ilog(\frac{exp^{(\frac{-1}{2})(x^t -m_i)^T\Sigma^{-1}(x^t-m_i)}}{2\pi^{\frac{N}{2}}|\Sigma|^{\frac{1}{2}}})$

$=∇_{\Sigma}\sum_t\sum_i h^t_i [\frac{-1}{2})(x^t -m_i)^T\Sigma^{-1}(x^t-m_i)]+\Sigma_ih^t_i {\frac{-1}{2}} log[|\Sigma|] =0$



$S^{l+1}_i=\frac{\sum^t h^t_i(x^t −m^{l+1}_i )(x^t −m^{l+1}_i )^T}{\sum_t h^t_i }$ ..............(7)

where, for Gaussian components in the E-step, we calculate

$h^t_i=\frac{π_i|S_i|^{−1/2 }exp[−(1/2)(x^t −m_i)^T S^{−1}_i (x^t −m_i)]} {\sum_j π_j |S_j |^{−1/2} exp[−(1/(x^t−m_j)^T S^{−1}_j (x^t −m_j)]}$ .........(8)




## DIFFERENT SCENARIOS

***I. Shared COVARIANCE MATRIX***

Lagrangian multiplier for maximisation becomes

$∇_{\Sigma}\sum_t\sum_i h^t_i [\frac{-1}{2})(x^t -m_i)^T\Sigma^{-1}(x^t-m_i)] =0$

$∇_{\Sigma}\sum_t\sum_i h^t_i [\frac{-1}{2})(x^t -m_i)^T\S^{-1}(x^t-m_i)] =0$


***II. Shared diagonal covariance Matrix***

Lagrangian multiplier for maximisation becomes


$∇_{\Sigma}\sum_t\sum_i h^t_i [\frac{-1}{2})(x^t -m_i)^T\Sigma^{-1}(x^t-m_i)] =0$

$∇_{\Sigma}\sum_t\sum_i h^t_i [\frac{-1}{2})(x^t -m_i)^T\ S^{-1}(x^t-m_i)] =0$

$∇_{\Sigma}\sum_t\sum_i h^t_i [\frac{-1}{2})(x^t -m_i)^2\ S^{-1}] =0$

$∇_{\Sigma}\sum_t\sum_i h^t_i [\frac{||x^t -m_i||^2}{s^2})] =0$

Estimation of Z becomes


$h^t_i= \frac{exp[−(1/2s^2)||x_t −m_i||^2]}{\Sigma_j exp[−(1/2s^2)||x_t −m_j||^2]}$

***Shapes of Clusters***

1. Kmeans cluster shapes are circular
2. EM algorithmn cluster shapes are ellipses of arbitary shapes,orientation and coverage


